# 3 Data Preprocessing<a id='3_Data_Preprocessing'></a>

## 3.1 Contents<a id='2.1_Contents'></a>
* [2 Data wrangling](#2_Data_wrangling)
  * [2.1 Contents](#2.1_Contents)
  * [2.2 Introduction](#2.2_Introduction)
    * [2.2.1 Recap Of Data Science Problem](#2.2.1_Recap_Of_Data_Science_Problem)
    * [2.2.2 Objective of Data Wrangling](#2.2.2_Objective_of_Data_Wrangling)
  * [2.3 Imports](#2.3_Imports)
  * [2.4 Load The Data](#2.4_Load_The_Data)
  * [2.5 Explore The Data](#2.5_Explore_The_Data)
    * [2.5.1 Find Duplicates](#2.5.1_Find_Duplicates)
    * [2.5.2 Find Missing Values](#2.5.2_Find_Missing_Values)
    * [2.5.3 Find Dimensions for Time Series](#2.5.3_Find_Dimensions_for_Time_Series)
      * [2.5.3.1 Total Number of Time Series](#2.5.3.1_Total_Number_of_Time_Series)
      * [2.5.3.2 Time Span of Time Series](#2.5.3.2_Time_Span_of_Time_Series)
        * [2.5.3.2.1 Time Span of Time Series in Weekly_sales Dataframe](#2.5.3.2.1_Time_Span_of_Time_Series_in_Weekly_sales_Dataframe)
        * [2.5.3.2.2 Time Span of Time Series in Features Dataframe](#2.5.3.2.2_Time_Span_of_Time_Series_in_Features _Dataframe)
    * [2.5.4 Time Trend of Weekly Sales](#2.5.4_Time_Trend_of_Weekly_Sales)
      * [2.5.4.1 Time Trend of Weekly Sales for Dept](#2.5.4.1_Time_Trend_of_Weekly_Sales_for_Dept)
      * [2.5.4.2 Completeness of Weekly Sales Records](#2.5.4.2_Completeness_of_Weekly_Sales_Records)
    * [2.5.5 Categorical Features](#2.5.5_Categorical_Features)
      * [2.5.5.1 Store Type](#2.5.5.1_Store_Type)
        * [2.5.5.1.1 Store Size for Different Store Type](#2.5.5.1.1_Store_Size_for_Different_Store_Type)
        * [2.5.5.1.2 Weekly Sales for Different Store Type](#2.5.5.1.2_Weekly_Sales_for_Different_Store_Type)
      * [2.5.5.2 Holiday Effect](#2.5.5.2_Holiday_Effect)
    * [2.5.6 Numerical Features](#2.5.6_Numerical_Features)
      * [2.5.6.1 Numerical Variables in features df](#2.5.6.1_Numerical_Variables_in_Features_df)
        * [2.5.6.1.1 Numerical Variables Summary](#2.5.6.1.1_Numerical_Variables_Summary)
        * [2.5.6.1.2 Numerical Variables Distribution](#2.5.6.1.2_Numerical_Variables_Distribution)
      * [2.5.6.2 Numerical Variables in weekly_sales_type df](#2.5.6.2_Numerical_Variables_in_Weekly_Sales_Type_df)
  * [2.6 Merge The Data](#2.6_Merge_The_Data)
  * [2.7 Save The dData](#2.7_Save_The_data)
  * [2.8 Summary](#2.8_Summary)


## 3.2 Objectives<a id='3.2_Objectives'></a>

2. Image preprocessing 
    
    1) Image normalization due to various image intensity 
    
    2) noise reduction technique - guassian blurring
    
    3) image augmentation
    
    4) ROI selection?
    

3. feature extraction
    1) statistical features like intensity mean, standard deviation, skewness, and kurtosis from the pixel intensity distributions.
 
    2) Shape and Contour Features: Hough Transform for detecting shapes in the X-ray which might be indicative of particular conditions.
    
    3) texture/orientation features: Gabor Filters
    
    4) deep learning CNNs
  
4. sample imbalance
   1) number in different cases are largely different, data imbalance might be a concern
    

In [ ]:
pip install keras-tuner

## 3.3 Imports<a id='3.3_Imports'></a>

In [1]:
#import libs  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense, Flatten,BatchNormalization, Conv2D,MaxPool2D,Dropout, Rescaling
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner

from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob


import seaborn as sns
import os

import cv2
from sklearn.model_selection import train_test_split



## 3.4 Test_Train_split<a id='3.4_Test_Train_split'></a>

In [2]:
categories = ['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity']  # List of categories

'''
def select_files(folder,proportion):
    files=[f for f in os.listdir(folder) if f.lower().endswith('.png','.jpg','jpeg')]
    num_selected_files=int(len(files)*proportion)
    selected_files=np.random.cjoice()files,num_selected_files,replace=False)
    return selected_files
'''

# Set a random seed for reproducibility
seed = 42

# Define a split ratio
train_ratio = 0.7
validation_ratio = 0.1
test_ratio = 0.2 

# Define the base directories for train and test sets
base_train_dir = 'train'
base_validation_dir = 'validation'
base_test_dir = 'test'

# Create train and test directories if they don't exist
if not os.path.exists(base_train_dir):
    os.makedirs(base_train_dir)
    
if not os.path.exists(base_validation_dir):
    os.makedirs(base_validation_dir)

if not os.path.exists(base_test_dir):
    os.makedirs(base_test_dir)

for category in categories:
    # Create category subdirectories in train and test directories
    train_category_dir = os.path.join(base_train_dir, category)
    validation_category_dir = os.path.join(base_validation_dir, category)
    test_category_dir = os.path.join(base_test_dir, category)

    if not os.path.exists(train_category_dir):
        os.makedirs(train_category_dir)
        
    if not os.path.exists(validation_category_dir):
        os.makedirs(validation_category_dir)
    
    if not os.path.exists(test_category_dir):
        os.makedirs(test_category_dir)

    # Get the list of images from the first subdirectory of the category
    subfolders = [d for d in os.listdir(category) if os.path.isdir(os.path.join(category, d))]  
    first_subfolder = os.path.join(category, subfolders[0])
    if os.path.isdir(first_subfolder):
        images = [img for img in os.listdir(first_subfolder) if img.endswith('.png')]

    # Train-test split for the current category
     # Select only 10% of images to use for this category
    _, use_images = train_test_split(images, test_size=0.3, random_state=seed)
    train_val_images, test_images = train_test_split(use_images, test_size=test_ratio, random_state=seed)
    train_images, val_images = train_test_split(train_val_images, test_size=validation_ratio/(train_ratio+validation_ratio), random_state=seed)

    # Move images to their respective train/test category directories
    for img in train_images:
        source = os.path.join(category, subfolders[0], img)
        destination = os.path.join(train_category_dir, img)
        shutil.move(source, destination)
        
    for img in val_images:
        source = os.path.join(category, subfolders[0], img)
        destination = os.path.join(validation_category_dir, img)
        shutil.move(source, destination)

    for img in test_images:
        source = os.path.join(category, subfolders[0], img)
        destination = os.path.join(test_category_dir, img)
        shutil.move(source, destination)



## 3.5 Baseline Model

### 3.5.1 Image preprocessing

In [3]:
#generate batches of data which can be put in sequential model 

#you can add augmentation or rescaling parameters here
#Frist, start without setting any parameters, since basic model already add rescaling function
Image_generator=ImageDataGenerator(rescale=1./255)

#creat batches of data from directory where train,val,test dataset are, target_size are set as 224 since pre-trained model VGG16 is using image with size of 224
train_batches=Image_generator.flow_from_directory(directory=base_train_dir,target_size=(299,299), classes=['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity'],batch_size=10)
val_batches=Image_generator.flow_from_directory(directory=base_validation_dir,target_size=(299,299), classes=['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity'],batch_size=10)
#only set shuffle to be false for test dataset since we need to use test dataset as reference for confusion matrix where
test_batches=Image_generator.flow_from_directory(directory=base_test_dir,target_size=(299,299), classes=['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity'],batch_size=10,shuffle=False)


Found 551 images belonging to 4 classes.
Found 82 images belonging to 4 classes.
Found 159 images belonging to 4 classes.


In [ ]:
#each batch contains 10 images and corresponding labels
imgs,labels=next(train_batches)

def adjustImages(image):
    min_val=np.min(img)
    max_val=np.max(img)
    adjusted_img=(img-min_val)/(max_val-min_val)
    return adjusted_img

In [ ]:
num_images=imgs.shape[0]

In [ ]:
num_images

In [ ]:
fig,axes=plt.subplots(1,num_images,figsize=(20,10))

    
for i,ax in enumerate(axes):
        
    ax.imshow(imgs[i])
    ax.axis('off')
    plt.tight_layout()
#    ax.set_title(f"label:{np.argmax(label[i])}")
plt.tight_layout()
plt.show()

### 3.5.2 Basic CNN model

In [ ]:
#build baseline model with minimal preprocessing by applying only normalization to the images
##input_shape=() must be in the first layer of the model, if the first layer is rescale=1./255, then add input_shape in this layer instead of conv2D
basic_CNN_model=Sequential([
                      Conv2D(filters=16, kernel_size=(3,3),activation='relu',padding='same',input_shape=(299,299,3)),
                      MaxPool2D(pool_size=(2,2),strides=2),
                      Conv2D(filters=32, kernel_size=(3,3),activation='relu'),
                      MaxPool2D(pool_size=(2,2),strides=2),
                      Flatten(),
                      Dense(units=4,activation='softmax')
                    ])
        #rescaling layer  to normaliza the input images, use 1./255 not becasue of image dimension, but because it's related to pixel intensity range, 
        #pixel intensity is represented by 8-bit value, range from 0-255, not the height by width of the image
        #padding is used to maintain the edge information, keep size of feature map same as input images
        #each filter is to detect specific type of feature, each filter will produce feature map
                      

In [ ]:
basic_CNN_model.summary()

In [ ]:
basic_CNN_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
basic_CNN_model.fit(x=train_batches,validation_data=val_batches,epochs=10,verbose=1)

### 3.5.3 Predict the Image 

In [ ]:
test_batches.classes

In [ ]:
test_imgs,test_labels=next(test_batches)

In [ ]:
fig,axes=plt.subplots(1,num_images,figsize=(20,5))

    
for i,ax in enumerate(axes):
        
    ax.imshow(test_imgs[i])
    ax.axis('off')
    plt.tight_layout()
    ax.set_title(f"label:{test_labels[i]}")
plt.tight_layout()
plt.show()

In [ ]:
predictions=basic_CNN_model.predict(x=test_batches,verbose=0)

In [ ]:
np.round(predictions)[:10]

In [ ]:
confusion_matrix=confusion_matrix(y_true=test_batches.classes,y_pred=np.argmax(predictions,axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
class_names=['COVID','Normal','Viral Pneumonia','Lung_Opacity']

In [ ]:
def CM_plot(confusion_matrix, color):
    plt.figure(figsize=(10,8))
    sns.heatmap(confusion_matrix,annot=True,fmt='d',cmap=color,xticklabels=class_names,yticklabels=class_names)
    plt.title('Confusion Matrix')
   
    #tick_marks=np.arrange(len(classes))
    plt.xlabel=('True Label')
    plt.ylabel=('Predicted Label')
    plt.show()
    
   

In [ ]:
CM_plot(confusion_matrix, "Blues")

# 3.6 CNN model with Image Augmentation

### 3.6.1 Image Augmentation

In [ ]:
#generate batches of data which can be put in sequential model 

#you can add augmentation or rescaling parameters here
#Frist, only apply augmentation to the training dataset

#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8048393/
    
Train_Image_generator=ImageDataGenerator(rescale=1./255,rotation_range=15,zoom_range=0.1,horizontal_flip=True, width_shift_range=0.1,brightness_range=[0.9,1.1],fill_mode='nearest')

#not apply to test dataset, only rescale the validation/test dataset
Image_generator_rescale=ImageDataGenerator(rescale=1./255)
#creat batches of data from directory where train,val,test dataset are, target_size are set as 224 since pre-trained model VGG16 is using image with size of 224
train_batches=Train_Image_generator.flow_from_directory(directory=base_train_dir,target_size=(299,299), classes=['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity'],batch_size=10)
val_batches=Image_generator_rescale.flow_from_directory(directory=base_validation_dir,target_size=(299,299), classes=['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity'],batch_size=10)
#only set shuffle to be false for test dataset since we need to use test dataset as reference for confusion matrix where
test_batches=Image_generator_rescale.flow_from_directory(directory=base_test_dir,target_size=(299,299), classes=['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity'],batch_size=10,shuffle=False)


In [ ]:
Aug_CNN_model=Sequential([Conv2D(filters=16, kernel_size=(3,3),activation='relu',padding='same',input_shape=(224,224,3)),
                      MaxPool2D(pool_size=(2,2),strides=2),
                      Conv2D(filters=32, kernel_size=(3,3),activation='relu',padding='same'),
                      MaxPool2D(pool_size=(2,2),strides=2),
                      Flatten(),
                      Dense(units=4,activation='softmax')])

In [4]:
def fit_model(model,l_r,train,val,ep,vb):
    model.compile(optimizer=Adam(learning_rate=l_r), loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(x=train,validation_data=val,epochs=ep,verbose=vb)
    return model

In [ ]:
def pred(model,x_test,vb):
    predictions=model.predict(x=x_test,verbose=vb)
    return predictions
    

In [ ]:
def CM_plot(predictions,test_batches,color):
    confusion_matrix=confusion_matrix(y_true=test_batches.classes,y_pred=np.argmax(predictions,axis=1))
    plt.figure(figsize=(10,8))
    sns.heatmap(confusion_matrix,annot=True,fmt='d',cmap=color,xticklabels=class_names,yticklabels=class_names)
    plt.title('Confusion Matrix')
   
    #tick_marks=np.arrange(len(classes))
    plt.xlabel=('True Label')
    plt.ylabel=('Predicted Label')
    plt.show()
    
    

### 3.6.2 hyperparameter tuning for augmentation

In [5]:
#1. define grid of parameters
rotation_values=[0,5,10,20]
zoom_values=[0,0.1,0,2]
width_shift_values=[0,0.1]
horizontal_flip=[True,False]

augmentation_configs = []
for width_shift in width_shift_values:
    for rotation in rotation_values:
        for zoom_range in zoom_values:
            config_name = f"ws_{width_shift}_rot_{rotation}_zoom_{zoom_range}"
            augmentation_configs.append({
                'name': config_name,
                'rotation_range': rotation,
                'width_shift_range': width_shift,
                'zoom_range':zoom_range,
                'horizontal_flip':horizontal_flip
       })

In [ ]:
def grid_search_augmentation(config,model,train_dir,val_dir,ep):
    train_data_generator=ImageDataGenerator(rescale=1./255,rotation_range=config['rotation_range'],width_shift_range=config['width_shift_range'],
                                           zoom_range=config['zoom_range'],horizontal_flip=config['horizontal_flip'],fill_mode='nearest')
    val_data_generator=ImageDataGenerator(rescale=1./255)
    
    train_batches=train_data_generator.flow_from_directory(directory=train_dir,target_size=(299,299), classes=['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity'],batch_size=10)
    val_batches=val_data_generator.flow_from_directory(directory=val_dir,target_size=(299,299), classes=['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity'],batch_size=10)
    model=model
    
    history=model.fit(train_batches,epochs=ep,validation_data=val_batches,vervose=2)
    best_val_accuracy=max(history.history['val_accuracy'])
    return best_val_accuracy


In [ ]:
results=[]
for config in augmentation_configs:
    accuracy=grid_search_augmentation(config,model,train,validation)
    results.append('configure':config['name'],'Accuracy':accuracy)

In [ ]:
results_df = pd.DataFrame(results)
# Display the DataFrame sorted by Accuracy
print(results_df.sort_values(by='Accuracy', ascending=False))